In [1]:
import pandas as pd 
preop_imaging_df = pd.read_excel('Data/Clean_imaging_data.xlsx')

**Function that calculates the sensitivity for each respective imaging modality** 

### Parameters
- **df**: the dataframe with imaging and concordance data  
- **modality_name**: the name of the scan column (e.g. `'4D CT Scan'`)

### Returns
- **sensitivity**: true positives / (true positives + false negatives)  
- **true_positives**: scans that matched surgery (exact or side)  
- **false_negatives**: scans that were non-localizing but surgery found disease



In [19]:
def calculate_sensitivity(df, modality_name):
    
    # initialize the concordance columns
    concordance_col = f"{modality_name} Calculated Concordance"
    
    # define true positives as exact or side matches 
    localized_results = df[df[concordance_col].isin(['Yes (Exact)', 'Yes (Side)'])]
    
    # define false negatives as non-localizing scan but sugery found disease 
    nonlocalized_results = df[df[modality_name] == 'Non-localizing']
    
    # calculate the total true positives and false negatives 
    true_positives = len(localized_results)
    false_negatives = len(nonlocalized_results)
    
    # avoid dividing by 0 
    if true_positives + false_negatives == 0:
        return None, 0, 0
    
    # define and calculate sensitivity 
    sensitivity = true_positives / (true_positives + false_negatives)
    return sensitivity, true_positives, false_negatives

In [20]:
# test the function to make sure it works 
sensitivity, true_positives, false_negatives = calculate_sensitivity(preop_imaging_df, '4D CT Scan')
print(f"4D CT Scan Sensitivity: {sensitivity:.2%} ({true_positives} TP / {true_positives + false_negatives} total)")


4D CT Scan Sensitivity: 72.64% (146 TP / 201 total)


In [21]:
# define the list of modalities 
modalities = ['SPECT/CT', '4D CT Scan', 'Ultrasound', 'Sestamibi', 'MRI']

# initialize a list to store the results 
results = []

# iterate through each individual modality 
for modality in modalities:
   
    # call the sensitivity function 
    sensitivity, true_positives, false_negatives = calculate_sensitivity(preop_imaging_df, modality)
    
    # make sure it isn't null
    if sensitivity is not None:
        
        # append each result to its respective modality
         results.append({
            'Modality': modality,
            # round the sensitivity
            'Sensitivity': round(sensitivity, 4),  
            'True Positives': true_positives,
            'False Negatives': false_negatives,
            'Total with Disease': true_positives + false_negatives
        })

In [ ]:
# Convert the list of dicts to a DataFrame
sensitivity_df = pd.DataFrame(results)



In [ ]:
sensitivity_df

,Modality,Sensitivity,True Positives,False Negatives,Total with Disease
0,SPECT/CT,0.7581,163,52,215
1,4D CT Scan,0.7264,146,55,201
2,Ultrasound,0.5000,67,67,134
3,Sestamibi,0.6474,112,61,173
4,MRI,0.4615,6,7,13


In [27]:
# create a new column that formats the sensitivity as a percentage 
sensitivity_df['Sensitivity (%)'] = (sensitivity_df['Sensitivity'] * 100).round(2)

# move that column to the front
sensitivity_df = sensitivity_df[['Modality', 'Sensitivity (%)', 'True Positives', 'False Negatives', 'Total with Disease']]